In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [3]:
import faiss 
import numpy as np
import pickle

In [4]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [5]:
class semanticEmbedding:
    def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    def get_embedding(self, sentences):
    # Tokenize sentences
        encoded_input = self.tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        # Perform pooling
        sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])

        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings.detach().numpy()

In [6]:
class FaissIdx:

    def __init__(self, model, dim=768):
        self.index = faiss.IndexFlatIP(dim) # need to load the pickle model in the final file
        # self.index = faiss
        # Maintaining the document data
        self.doc_map = dict()
        self.model = model
        self.ctr = 0

    def add_doc(self, document_text):
        self.index.add(self.model.get_embedding(document_text))
        self.doc_map[self.ctr] = document_text # store the original document text
        self.ctr += 1

    def search_doc(self, query, k=3):
        D, I = self.index.search(self.model.get_embedding(query), 5)
        return [{self.doc_map[idx]: score} for idx, score in zip(I[0], D[0]) if idx in self.doc_map]
    def save_index(self, index_filename, doc_map_filename):
        # Save Faiss index to file
        faiss.write_index(self.index, index_filename)

        # Save document map to file using pickle
        with open(doc_map_filename, 'wb') as f:
            pickle.dump(self.doc_map, f)

    def load_index(self, index_filename, doc_map_filename):
        # Load Faiss index from file
        self.index = faiss.read_index(index_filename)

        # Load document map from file using pickle
        with open(doc_map_filename, 'rb') as f:
            self.doc_map = pickle.load(f)

In [7]:
# creating an instance of the class
model = semanticEmbedding()

In [8]:
index = FaissIdx(model)

In [9]:
index.load_index('index.bin', 'doc_map.pkl')

In [10]:
import google.generativeai as genai

In [11]:
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

In [12]:
model = genai.GenerativeModel('gemini-pro')

# given the query, run the following code sequentially

In [37]:
query = "I want to improve the situtaion of rural womens"

In [47]:
query_result = index.search_doc(query)

In [39]:
context = ''

In [40]:
for i in range(5):
    for key, value in query_result[i].items():
        if(value>0.23):
            context += key

In [41]:
context

'Information on Mahila Samakhya Programme by Department of School Education and Literacy = Mahila Samakhya Programme was launched for the education and empowerment of women in rural areas. Users can get information about Mahila Samakhya Programme like funding pattern, genesis, expansion & coverage and plan document. Details of women mobilisation and programme agendas and role of the National Resource Group of Mahila Samakhya are provided. Information about National Evaluation - Executive Summary in different states are also provided. Links to other Mahila Samakhya Sites are provided.Women Empowerment Scheme, Ministry of Women & Child Development = Get Information on Women Empowerment Scheme provided by the Ministry of Women & Child Development. Ministry of Women and Child Development, Government of India have come up with various schemes, programmes, social welfare schemes, Health and Nutrition, scholarship for women empowerment, Girl Child pregnant women, mothers, ward members, Anganw

In [42]:
content = ""

In [43]:
if context == '':
    print('Is there anything else I can help you with')
else:
    content = f'Use this query and respond with the following context in a presentatble manner. QUERY:{query} CONTEXT:{context}'

In [44]:
content

'Use this query and respond with the following context in a presentatble manner. QUERY:I want to improve the situtaion of rural womens CONTEXT:Information on Mahila Samakhya Programme by Department of School Education and Literacy = Mahila Samakhya Programme was launched for the education and empowerment of women in rural areas. Users can get information about Mahila Samakhya Programme like funding pattern, genesis, expansion & coverage and plan document. Details of women mobilisation and programme agendas and role of the National Resource Group of Mahila Samakhya are provided. Information about National Evaluation - Executive Summary in different states are also provided. Links to other Mahila Samakhya Sites are provided.Women Empowerment Scheme, Ministry of Women & Child Development = Get Information on Women Empowerment Scheme provided by the Ministry of Women & Child Development. Ministry of Women and Child Development, Government of India have come up with various schemes, program

In [45]:
response = model.generate_content(content)

In [46]:
print(response.text)

**Improving the Situation of Rural Women through Mahila Samakhya Programme**

**Mahila Samakhya Programme Overview:**

The Mahila Samakhya Programme is a government initiative launched to empower and educate women in rural areas.

**Key Features:**

* **Funding:** The programme receives funding from the Department of School Education and Literacy.
* **Expansion:** The programme has been expanded to cover various states and districts.
* **Genesis:** The programme was established to address the educational and social disparities faced by rural women.
* **Women Mobilization:** The programme focuses on mobilizing women and encouraging their participation in community development.
* **National Resource Group:** The National Resource Group provides guidance and support to Mahila Samakhya members.
* **National Evaluation:** Regular evaluations assess the effectiveness of the programme in different states.

**Other Related Schemes and Programmes:**

* **Women Empowerment Scheme:** This scheme 